## **Algunos algoritmos usados en PyOD**

#### **Angle-Based Outlier Detection (ABOD)**

* Considera la relación entre cada punto y su(s) vecino(s). No considera las relaciones entre estos vecinos. El anomaly score es la varianza de las distancias coseno a los vecinos

* ABOD funciona bien con datos multidimensionales
* PyOD ofrece dos versiones diferentes de ABOD:
    * Fast ABOD: Utiliza los k vecinos más próximos
    * ABOD original: Considera todos los puntos de training

#### **k-Nearest Neighbors**

* El anomaly score es función de las distancias a los vecinos más cercanos
* PyOD tiene tres versiones de kNN:
    * Max: Utiliza la distancia al k-ésimo vecino
    * Average: Utiliza la media de las distancias a todos los k vecinos
    * Median: Utiliza la mediana de las distancias a los k vecinos

#### **Isolation Forest**

* Utiliza internamente la biblioteca scikit-learn.
* El análisis se realiza mediante un conjunto de árboles.
* Isolation Forest proporciona un anomaly score que analiza el grado de aislamiento del punto en la estructura.
* Isolation Forest funciona bien con datos multidimensionales

#### **Histogram-based Outlier Detection**

* Es un método eficiente no supervisado que asume la independencia de las características y calcula el anomaly score mediante histogramas.
* Es mucho más rápido que los enfoques multivariantes, pero a costa de una menor precisión.

#### **Local Correlation Integral (LOCI)**

* LOCI es muy eficaz para detectar valores atípicos y grupos de valores atípicos. Proporciona un gráfico LOCI para cada punto que resume gran parte de la información sobre los datos en el área que rodean al punto determinando los clusters, los microclusters, sus diámetros y las distancias entre clusters.
* Ninguno de los métodos de detección de anomalías puede igualar esta característica, ya que sólo proporcionan un único score para cada punto.

#### **Feature Bagging**

* El feature bagging construye un conjunto de detectores usando diferentes muestras del conjunto de datos. Las diferentes muestras se obtienen usando un conjunto diferente de variables. Utiliza el promedio u otros métodos de combinación para mejorar la precisión de la predicción.
* Por defecto, se utiliza el Factor de Anomalía Local (LOF) como estimador base. Sin embargo, se puede utilizar cualquier estimador como base, por ejemplo kNN y ABOD.

#### **Clustering Based Local Outlier Factor**

* Clasifica los datos en clusters pequeños y clusters grandes. El anomaly score se calcula en función del tamaño del clúster al que pertenece el punto, así como de la distancia al clúster grande más cercano.

## Librería PyOD en dataset sintético

In [ ]:
!pip install pyod -q

In [ ]:
COLAB = True

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1Hv4IDdaAB3nOhTwsXPeiRLUmwnkJ4fEb',
                                        dest_path='./big_mart_sales_training.csv')

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager

In [ ]:
from pyod.models.abod import ABOD
from pyod.models.knn import KNN

In [ ]:
from pyod.utils.data import generate_data, get_outliers_inliers

# generate random data with two features
# generate_data: Inliers data is generated by a multivariate Gaussian distribution
#                Outliers are generated by a uniform distribution
X_train, Y_train = generate_data(n_train=200,train_only=True, n_features=2)

# by default the outlier fraction is 0.1 in generate data function 
outlier_fraction = 0.1

# store outliers and inliers in different numpy arrays
x_outliers, x_inliers = get_outliers_inliers(X_train,Y_train)

n_inliers = len(x_inliers)
n_outliers = len(x_outliers)

#separate the two features and use it to plot the data 
F1 = X_train[:,[0]].reshape(-1,1)
F2 = X_train[:,[1]].reshape(-1,1)

# create a meshgrid 
xx , yy = np.meshgrid(np.linspace(-10, 10, 200), np.linspace(-10, 10, 200))

# scatter plot 
plt.scatter(F1,F2)
plt.xlabel('F1')
plt.ylabel('F2');

In [ ]:
# clasificadores que se usarán

classifiers = {
     'Angle-based Outlier Detector (ABOD)'   : ABOD(contamination=outlier_fraction),
     'K Nearest Neighbors (KNN)' :  KNN(contamination=outlier_fraction)
}

In [ ]:
#set the figure size
plt.figure(figsize=(10, 5))

for i, (clf_name,clf) in enumerate(classifiers.items()) :
    # fit the dataset to the model
    clf.fit(X_train)

    # predict raw anomaly score
    scores_pred = clf.decision_function(X_train)*-1

    # prediction of a datapoint category outlier or inlier
    y_pred = clf.predict(X_train)

    # no of errors in prediction
    n_errors = (y_pred != Y_train).sum()
    print('No of Errors : ',clf_name, n_errors)

    # rest of the code is to create the visualization

    # threshold value to consider a datapoint inlier or outlier
    threshold = stats.scoreatpercentile(scores_pred,100 *outlier_fraction)

    # decision function calculates the raw anomaly score for every point
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
    Z = Z.reshape(xx.shape)

    subplot = plt.subplot(1, 2, i + 1)

    # fill blue colormap from minimum anomaly score to threshold value
    subplot.contourf(xx, yy, Z, levels = np.linspace(Z.min(), threshold, 10),cmap=plt.cm.Blues_r)

    # draw red contour line where anomaly score is equal to threshold
    a = subplot.contour(xx, yy, Z, levels=[threshold],linewidths=2, colors='red')

    # fill orange contour lines where range of anomaly score is from threshold to maximum anomaly score
    subplot.contourf(xx, yy, Z, levels=[threshold, Z.max()],colors='orange')

    # scatter plot of inliers with white dots
    b = subplot.scatter(X_train[:-n_outliers, 0], X_train[:-n_outliers, 1], c='white',s=20, edgecolor='k') 
    # scatter plot of outliers with black dots
    c = subplot.scatter(X_train[-n_outliers:, 0], X_train[-n_outliers:, 1], c='black',s=20, edgecolor='k')
    subplot.axis('tight')

    subplot.legend(
        [a.collections[0], b, c],
        ['learned decision function', 'true inliers', 'true outliers'],
        prop=matplotlib.font_manager.FontProperties(size=10),
        loc='lower right')

    subplot.set_title(clf_name)
    subplot.set_xlim((-10, 10))
    subplot.set_ylim((-10, 10))
plt.show()

## Librería PyOD en dataset de ventas de Big Mart

In [ ]:
!pip install combo -q

In [ ]:
import pandas as pd

# Import models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF

In [ ]:
df = pd.read_csv("./big_mart_sales_training.csv")

In [ ]:
df.plot.scatter('Item_MRP','Item_Outlet_Sales', alpha=0.1);

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
df[['Item_MRP','Item_Outlet_Sales']] = scaler.fit_transform(df[['Item_MRP','Item_Outlet_Sales']])
df[['Item_MRP','Item_Outlet_Sales']].head()

In [ ]:
X1 = df['Item_MRP'].values.reshape(-1,1)
X2 = df['Item_Outlet_Sales'].values.reshape(-1,1)

X = np.concatenate((X1,X2),axis=1)

In [ ]:
random_state = np.random.RandomState(42)
outliers_fraction = 0.05
# Define seven outlier detection tools to be compared
classifiers = {
        'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor (CBLOF)':CBLOF(contamination=outliers_fraction,check_estimator=False, random_state=random_state),
        'Feature Bagging':FeatureBagging(LOF(n_neighbors=35),contamination=outliers_fraction,check_estimator=False,random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Average KNN': KNN(method='mean',contamination=outliers_fraction)
}

In [ ]:
xx , yy = np.meshgrid(np.linspace(0,1 , 200), np.linspace(0, 1, 200))

for i, (clf_name, clf) in enumerate(classifiers.items()):
    clf.fit(X)
    # predict raw anomaly score
    scores_pred = clf.decision_function(X) * -1
        
    # prediction of a datapoint category outlier or inlier
    y_pred = clf.predict(X)
    n_inliers = len(y_pred) - np.count_nonzero(y_pred)
    n_outliers = np.count_nonzero(y_pred == 1)
    plt.figure(figsize=(10, 10))
    
    # copy of dataframe
    dfx = df
    dfx['outlier'] = y_pred.tolist()
    
    # IX1 - inlier feature 1,  IX2 - inlier feature 2
    IX1 =  np.array(dfx['Item_MRP'][dfx['outlier'] == 0]).reshape(-1,1)
    IX2 =  np.array(dfx['Item_Outlet_Sales'][dfx['outlier'] == 0]).reshape(-1,1)
    
    # OX1 - outlier feature 1, OX2 - outlier feature 2
    OX1 =  dfx['Item_MRP'][dfx['outlier'] == 1].values.reshape(-1,1)
    OX2 =  dfx['Item_Outlet_Sales'][dfx['outlier'] == 1].values.reshape(-1,1)
         
    print('OUTLIERS : ',n_outliers,'INLIERS : ',n_inliers, clf_name)
        
    # threshold value to consider a datapoint inlier or outlier
    threshold = stats.scoreatpercentile(scores_pred,100 * outliers_fraction)
        
    # decision function calculates the raw anomaly score for every point
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()]) * -1
    Z = Z.reshape(xx.shape)
          
    # fill blue map colormap from minimum anomaly score to threshold value
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),cmap=plt.cm.Blues_r)
        
    # draw red contour line where anomaly score is equal to thresold
    a = plt.contour(xx, yy, Z, levels=[threshold],linewidths=2, colors='red')
        
    # fill orange contour lines where range of anomaly score is from threshold to maximum anomaly score
    plt.contourf(xx, yy, Z, levels=[threshold, Z.max()],colors='orange')
        
    b = plt.scatter(IX1,IX2, c='white',s=20, edgecolor='k')
    
    c = plt.scatter(OX1,OX2, c='black',s=20, edgecolor='k')
       
    plt.axis('tight')  
    
    # loc=2 is used for the top left corner 
    plt.legend(
        [a.collections[0], b,c],
        ['learned decision function', 'inliers','outliers'],
        prop=matplotlib.font_manager.FontProperties(size=20),
        loc=2)
      
    plt.xlim((0, 1))
    plt.ylim((0, 1))
    plt.title(clf_name, fontsize=20)
    plt.show()